In [1]:
%pip install numpy scikit-learn


Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


### Resolución

#### 1. Resolución

In [22]:
def calcular_similitudes_documentos(n_docs=5, mostrar_texto=True):
    """
    Analiza la similitud entre documentos usando similaridad coseno.
    
    Parámetros:
    n_docs (int): Cantidad de documentos a analizar
    mostrar_texto (bool): Si True muestra extractos de texto en los resultados
    """
    
    # 1. Vectorización de documentos 
    tfidfvect = TfidfVectorizer()
    X_train = tfidfvect.fit_transform(newsgroups_train.data)
    
    # 2. Selección aleatoria de documentos
    np.random.seed(142)  # Para reproducibilidad
    random_indices = np.random.choice(X_train.shape[0], size=n_docs, replace=False)
    i = 1

    # 3. Cálculo de similitudes para cada documento seleccionado
    for idx in random_indices:
        # Calcular similaridad con todos los documentos
        cossim = cosine_similarity(X_train[idx], X_train).flatten()
        
        #print(np.sort(cossim)[::-1])

        # Obtener índices de los 5 más similares (excluyendo el propio documento)
        most_sim = np.argsort(cossim)[-6:-1][::-1]  # -6:-1 para excluir similitud consigo mismo #[::-1] invierte el contenido

        
        # 4. Análisis de resultados
        print(f"\n### Documento base numero {i} ({idx}) - Categoría: {newsgroups_train.target_names[y_train[idx]]}")
        if mostrar_texto:
            print("\nContenido:", newsgroups_train.data[idx][:250] + "...")  # Mostrar inicio del texto
        
        # Verificar categorías de documentos similares
        categorias_similares = [newsgroups_train.target_names[y_train[i]] for i in most_sim]
        print("\nDocumentos similares:")
        for sim_idx, categoria in zip(most_sim, categorias_similares):
            print(f"• Doc {sim_idx}: {categoria} ({cossim[sim_idx]:.3f})")
            if mostrar_texto:
                print(f"   Texto: {newsgroups_train.data[sim_idx][:200]}...\n")

        # Análisis de consistencia en las categorías
        coincidencias = sum(1 for cat in categorias_similares if cat == newsgroups_train.target_names[y_train[idx]])
        print(f"\nCoincidencias de categoría: {coincidencias}/5\n")
        print("="*80)
        i = i + 1
# Ejecutar análisis
calcular_similitudes_documentos()



### Documento base numero 1 (4698) - Categoría: soc.religion.christian

Contenido: There was a recent discussion of Dungeons and Dragons and other role
playing games.  Since there is a lot of crossover between gamers and
science fiction and fantasy fans, I will mention that I am the editor
and publisher of RADIO FREE THULCANRA, a C...

Documentos similares:
• Doc 10728: soc.religion.christian (0.236)
   Texto: <<Posting deleted.  The moderator replies:

That is generally accuate, but contains one serious error.  We Catholics
do believe that God's revealed truth that is not explicitly recorded in
the Bible c...

• Doc 3705: sci.med (0.219)
   Texto: 
To start with, no methodology or form of reasoning is infallible.  So
there's a question of how much certainty we are willing to pay for in a
given context.  Insistence on too much rigor bogs science...

• Doc 3216: soc.religion.christian (0.209)
   Texto: [DISCLAIMER: Throughout this post, there are statements and questions which
could ea

#### 2. Resolución

In [23]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Utilizo gridsearch para encontrar los mejores hyperparametros

# Pipeline para experimentar con diferentes configuraciones
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('classifier', MultinomialNB())
])

# Parámetros a probar para optimización
param_grid = [
    {
        'vectorizer__max_features': [5000, 10000, 30000],
        'vectorizer__ngram_range': [(1,1), (1,2),(1,3)],
        'vectorizer__stop_words': [None, 'english'],
        'classifier': [MultinomialNB(), ComplementNB()],
        'classifier__alpha': [0.1, 1.0, 10.0]
    }
]

# Búsqueda grid con validación cruzada
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=3,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)


In [24]:
grid_search.fit(newsgroups_train.data, newsgroups_train.target)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


/opt/anaconda3/envs/ia_ceia_18co/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                                       ('classifier', MultinomialNB())]),
             n_jobs=-1,
             param_grid=[{'classifier': [MultinomialNB(), ComplementNB()],
                          'classifier__alpha': [0.1, 1.0, 10.0],
                          'vectorizer__max_features': [5000, 10000, 30000],
                          'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
                          'vectorizer__stop_words': [None, 'english']}],
             scoring='f1_macro', verbose=1)

In [25]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(newsgroups_test.data)
print(f"F1-score macro: {f1_score(y_test, y_pred, average='macro'):.3f}")
print(f"Mejores parámetros: {grid_search.best_params_}")


F1-score macro: 0.693
Mejores parámetros: {'classifier': ComplementNB(), 'classifier__alpha': 1.0, 'vectorizer__max_features': 30000, 'vectorizer__ngram_range': (1, 1), 'vectorizer__stop_words': 'english'}


In [26]:
# Prueba rápida de modelos
models = {
    'MultinomialNB': MultinomialNB(alpha=0.1),
    'ComplementNB': ComplementNB(alpha=1.0)
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"{name} F1 Score: {f1_score(y_test, y_pred, average='macro'):.3f}")


MultinomialNB F1 Score: 0.656
ComplementNB F1 Score: 0.693


In [27]:
## 3. Resolución

In [28]:
def similitud_palabras_manual(palabras_seleccionadas, mostrar_resultados=True):
    """
    Calcula y muestra las palabras más similares a una lista de palabras seleccionadas manualmente,
    usando similaridad coseno en una matriz término-documento.
    
    Parámetros:
    palabras_seleccionadas (list): Lista de palabras a analizar (deben existir en el vocabulario)
    mostrar_resultados (bool): Si True, imprime los resultados formateados
    
    Retorna:
    dict: Diccionario con cada palabra y sus palabras similares ordenadas
    """
    
    # 1. Transponer matriz documento-término para obtener término-documento
    X_train_T = X_train.T  # Cada fila ahora representa una palabra
    
    # 2. Obtener índices de las palabras seleccionadas
    indices_palabras = [tfidfvect.vocabulary_[palabra] for palabra in palabras_seleccionadas]
    
    # 3. Calcular similaridad coseno entre palabras seleccionadas y todas las demás
    similitudes = cosine_similarity(X_train_T[indices_palabras], X_train_T)
    
    # 4. Encontrar las 5 palabras más similares para cada término
    resultados = {}
    for i, palabra in enumerate(palabras_seleccionadas):
        # Obtener índices ordenados por similitud (mayor a menor)
        indices_similitud = np.argsort(similitudes[i])[::-1]
        
        # Filtrar la propia palabra (similitud 1 consigo misma)
        indices_similitud = indices_similitud[indices_similitud != indices_palabras[i]]
        
        # Tomar las 5 primeras y mapear a palabras reales
        top5_indices = indices_similitud[:5]
        top5_palabras = [idx2word[idx] for idx in top5_indices]
        
        resultados[palabra] = top5_palabras
        
        # Mostrar resultados si está habilitado
        if mostrar_resultados:
            print(f"Palabra: {palabra}")
            print("Top 5 palabras similares:")
            for p in top5_palabras:
                print(f"  - {p}")
            print("\n" + "-"*60 + "\n")
    
    return resultados

# Selección manual de palabras relevantes (evitar términos ambiguos)
palabras_analizar = ['government', 'car', 'medical', 'space', 'religion']

# Ejecutar análisis
resultados_similitud = similitud_palabras_manual(palabras_analizar)


Palabra: government
Top 5 palabras similares:
  - the
  - to
  - of
  - libertarian
  - encryption

------------------------------------------------------------

Palabra: car
Top 5 palabras similares:
  - cars
  - criterium
  - civic
  - owner
  - dealer

------------------------------------------------------------

Palabra: medical
Top 5 palabras similares:
  - romano
  - hospitals
  - recuperation
  - providers
  - relelvant

------------------------------------------------------------

Palabra: space
Top 5 palabras similares:
  - nasa
  - seds
  - shuttle
  - enfant
  - seti

------------------------------------------------------------

Palabra: religion
Top 5 palabras similares:
  - religious
  - religions
  - categorized
  - purpsoe
  - crusades

------------------------------------------------------------

